In [ ]:
# Extract all committee names
import pandas as pd
import re
datasets = [f"data/US/{term*2+2009-222}-{term*2+2010-222}_{term}th_Congress/csv/bills.csv" for term in range(111,120)]
bills_df=pd.concat([pd.read_csv(dataset) for dataset in datasets])
committees=[committee for committee in set(bills_df["committee"]) if type(committee)==str and committee !=" " and committee!=""]
committee_spellings={committee:[committee] for committee in committees}
committee_search_item={}
for committee in committees:
  committee_spellings[committee]=[]
  if not re.search("committee",committee.lower()):
    committee_spellings[committee].append(committee + " Committee")
    committee_spellings[committee].append("Committee on "+committee)
    committee_spellings[committee].append("Committee on the "+committee)
    if re.search("Senate",committee):
      committee_spellings[committee].append(re.sub("Senate ","",committee)+ " Committee")
      committee_spellings[committee].append("Senate Committee on the "+re.sub("Senate ","",committee))
      committee_spellings[committee].append("Senate Committee on "+re.sub("House ","",committee))
      committee_spellings[committee].append("Committee on the "+re.sub("Senate ","",committee))
      committee_spellings[committee].append("Committee on "+re.sub("Senate ","",committee))
    if re.search("House",committee):
      committee_spellings[committee].append(re.sub("House ","",committee)+ " Committee")
      committee_spellings[committee].append("House Committee on the "+re.sub("House ","",committee))
      committee_spellings[committee].append("House Committee on "+re.sub("House ","",committee))
      committee_spellings[committee].append("Committee on the "+re.sub("House ","",committee))
      committee_spellings[committee].append("Committee on "+re.sub("House ","",committee))
  committee_spellings[committee].append(committee)
  if re.search("(House )|(Senate )",committee):
    committee_spellings[committee].append(re.sub("(House )|(Senate )","",committee))
  if re.search("committee",committee):
      committee_spellings[committee].append(re.sub("(Subcommittee on )|(Subcommittee for )|(Subcommittee )|( Subcommittee)|(Committee on )|(Committee for )|( Committee)|(Committee )","",committee))
  if re.search("committee",committee) and re.search("(House )|(Senate )",committee):
      committee_spellings[committee].append(re.sub("(Subcommittee on )|(Subcommittee for )|(Subcommittee )|( Subcommittee)|(Committee on )|(Committee for )|( Committee)|(Committee )|(Senate )|(House )","",committee))
  committee_search_item[committee]=re.sub("(Subcommittee on )|(Subcommittee for )|(Subcommittee )|( Subcommittee)|(Committee on )|(Committee for )|( Committee)|(Committee )|(Senate )|(House )","",committee)
#   committees.append("Committee on "+committee)

display(committee_spellings)
display(committee_search_item)
# for committee in committees:
#   print(committee)

#   committees.append("Committee on "+committee)

In [ ]:
# Map each action to the committee[s] it is in
import json
datasets = [f"data/US/{term*2+2009-222}-{term*2+2010-222}_{term}th_Congress/csv/history.csv" for term in range(111,120)]
history_df=pd.concat([pd.read_csv(dataset) for dataset in datasets])
actions = list(set(history_df["action"]))
def actions_to_committees(actions):
  action_committees_map={}
  for action in actions:
    action_committees_map[action]=[]
    for committee in committees:
      if re.search(committee_search_item[committee],action):
        action_committees_map[action].append(committee)
  return action_committees_map
action_committees_map=actions_to_committees(actions)
# display([x for x in action_committees_map.items() if len(x[1])>0])
json.dump(action_committees_map,open("./outputs/action_committees_map.json","w"))

In [ ]:
import json
action_committees_map=json.load(open("./outputs/action_committees_map.json","r"))

In [ ]:
# Functions for comparing two actions to see similarity
from collections import defaultdict, Counter
import re
import math
import numpy as np
# action_process_memo={}
def topsort(g):
  out=[]
  visited=set()
  def explore(node):
    if node not in visited:
      for neighbor in g[node]:
        explore(neighbor)
      out.append(node)
      visited.add(node)
  for node in g:
    explore(node)
  return list(reversed(out))
def longest_path(g):
  topsorting = topsort(g)
  longest_path_node={}
  for node in topsorting:
    if node not in longest_path_node:
      longest_path_node[node]=0
    for neighbor in g[node]:
      if neighbor not in longest_path_node:
        longest_path_node[neighbor]=0
      longest_path_node[neighbor]=max(longest_path_node[neighbor],longest_path_node[node]+1)
  return max([longest_path_node[x] for x in longest_path_node]) if len(longest_path_node)!=0 else 0

import itertools
action_process_map={}
def process_action(action):
  if action in action_process_map:
    return action_process_map[action]
  action=action.lower()
  action=re.sub("committees","committee",action)
  committees=[committee for committee in committee_search_item if re.search(committee_search_item[committee].lower(),action) and not re.search("subcommittee",action.lower())]
  subcommittees=[committee for committee in committee_search_item if re.search(committee_search_item[committee].lower(),action) and re.search("subcommittee",action.lower())]
  if len(committees)>0:
    committee_spellings_regex="|".join(["("+")|(".join(committee_spellings[committee])+")" for committee in committees]).lower()
    action = re.sub(committee_spellings_regex,"committee",action)
  if len(subcommittees)>0:
    committee_spellings_regex="|".join(["("+")|(".join(committee_spellings[committee])+")" for committee in subcommittees]).lower()
    action = re.sub(committee_spellings_regex,"subcommittee",action)
  action=re.sub(r"\,|\.|\-"," ",action)
  action=re.sub(r"[0-9]","",action)
  action=re.sub(r" +"," ",action)
  action = re.sub(r'\b(mr|mrs|ms|senator|representative) \w+\s', 'representative ',action)
  action = re.sub(r'(\w+ hour)|(\w+ minutes)', 'time',action)
  action = re.sub(r'\(.*?\)', '',action)
  action=re.sub(r" +"," ",action)
  action_process_map[action]=action
  return action
# def rename(action):
#   # if action in action_process_map:
#   #   return action_process_map[action]
#   # action=action.lower()
#   # action=re.sub("committees","committee",action)
#   # committees=[committee for committee in committee_search_item if re.search(committee_search_item[committee].lower(),action) and not re.search("subcommittee",action.lower())]
#   # subcommittees=[committee for committee in committee_search_item if re.search(committee_search_item[committee].lower(),action) and re.search("subcommittee",action.lower())]
#   # if len(committees)>0:
#   #   committee_spellings_regex="|".join(["("+")|(".join(committee_spellings[committee])+")" for committee in committees]).lower()
#   #   action = re.sub(committee_spellings_regex,"the committee",action)
#   # if len(subcommittees)>0:
#   #   committee_spellings_regex="|".join(["("+")|(".join(committee_spellings[committee])+")" for committee in subcommittees]).lower()
#   #   action = re.sub(committee_spellings_regex,"the subcommittee",action)
#   # action=re.sub()
#   # action=re.sub(r"\,|\.|\-"," ",action)
#   # action=re.sub(r"[0-9]","",action)
#   # action=re.sub(r" +"," ",action)
#   # action = re.sub(r'\b(mr|mrs|ms|senator|representative) \w+\s', 'representative ',action)
#   # action = re.sub(r'(\w+ hour)|(\w+ minutes)', 'time',action)
#   action = re.sub(r'\(.*?\)', '',action)
#   # action=re.sub(r" +"," ",action)
#   # action_process_map[action]=action
#   return action
action_process_split_map={}
def process_split_action(action):
  if action in action_process_split_map:
    return action_process_split_map[action]
  processed_action=process_action(action)
  processed_split_action=processed_action.split(" ")
  processed_split_action=[x for x in processed_split_action if x!=""]
  action_process_split_map[action]=processed_split_action
  return processed_split_action

def longest_common_subsequence_old(sentence1,sentence2):
  if len(sentence1)==0 or len(sentence2)==0:
    return 0
  matches=[]
  for i1,word1 in enumerate(sentence1):
    for i2,word2 in enumerate(sentence2):
      if word1==word2:
        matches.append((i1,i2))
  if len(matches)==0:
    return 0
  g = {}
  for match1 in matches:
    if match1 not in g:
      g[match1]=[]
    for match2 in matches:
      if match2 not in g:
        g[match2]=[]
      if match1[0]<match2[0] and match1[1]<match2[1]:
        g[match1].append(match2)
  return longest_path(g)+1
def edit_distance(l1,l2):
  dp=np.empty((len(l2)+1,len(l1)+1))
  dp[0]=range(len(l1)+1)
  dp[:,0]=range(len(l2)+1)
  for i in range(1,len(l2)+1):
    for j in range(1,len(l1)+1):
      dp[i,j]=min(dp[i][j-1]+1,dp[i-1][j]+1,dp[i-1][j-1] if l1[j-1]==l2[i-1] else dp[i-1][j-1]+1)
  return dp[-1,-1]
def longest_common_subsequence(l1, l2):
  # Helper function to calculate edit distance without substitutions allowed
  def edit_distance_no_subs(l1, l2):
    # Create dp matrix with size (len(l2)+1) x (len(l1)+1)
    dp = np.empty((len(l2)+1, len(l1)+1))
    # Initialize first row and column
    dp[0] = range(len(l1)+1)
    dp[:,0] = range(len(l2)+1)
    # Set infinity value for substitutions
    inf = float('inf')  # Use float('inf') instead of len(l1)*len(l2)
    
    # Fill dp matrix
    for i in range(1, len(l2)+1):
      for j in range(1, len(l1)+1):
        # If characters match, take diagonal value
        # If not, take minimum of insert/delete plus 1
        # Substitutions are not allowed (inf cost)
        dp[i,j] = min(
          dp[i][j-1] + 1,  # deletion
          dp[i-1][j] + 1,  # insertion
          dp[i-1][j-1] if l1[j-1] == l2[i-1] else inf  # match or substitution
        )
    return dp[-1][-1]

  # Handle empty sequences
  if not l1 or not l2:
    return 0
    
  # LCS length = (len(l1) + len(l2) - edit_distance_no_subs) / 2
  # Since each non-matching character requires one insertion and one deletion
  distance = edit_distance_no_subs(l1, l2)
  return (len(l1) + len(l2) - distance) // 2

def test_edit_distance():
  # Test empty strings
  assert edit_distance([], []) == 0
  # Test one empty string
  assert edit_distance([], ['a']) == 1
  assert edit_distance(['a'], []) == 1
  # Test single character difference
  assert edit_distance(['a'], ['b']) == 1
  # Test same strings
  assert edit_distance(['a', 'b', 'c'], ['a', 'b', 'c']) == 0
  # Test insertion
  assert edit_distance(['a', 'c'], ['a', 'b', 'c']) == 1
  # Test deletion
  assert edit_distance(['a', 'b', 'c'], ['a', 'c']) == 1
  # Test substitution
  assert edit_distance(['a', 'b', 'c'], ['a', 'd', 'c']) == 1
  # Test multiple operations
  assert edit_distance(['a', 'b'], ['c', 'd']) == 2

# test_edit_distance()
mustmatch_regexes=["subcommittee"]
def similar_words(action1,action2,threshold):
  for regex in mustmatch_regexes:
    if not (bool(re.search(regex,action1.lower() if regex.islower() else action1))==bool(re.search(regex,action2.lower() if regex.islower() else action2))):
      return False
  action1=process_split_action(action1)
  action2=process_split_action(action2)
  length=max(len(action1),len(action2))
  return abs(len(action1)-len(action2))<threshold*length and edit_distance_below(action1,action2,threshold*length)
def similar_string(action1,action2,threshold):
  action1=process_action(action1)
  action2=process_action(action2)

  for regex in mustmatch_regexes:
    if not (bool(re.search(regex,action1))==bool(re.search(regex,action2))):
      return False
  length=max(len(action1),len(action2))
  return abs(len(action1)-len(action2))<threshold*length and edit_distance_below(action1,action2,threshold*length)


# Test cases
# print("Test 1:", subsequence_similarity("hi my name is Bob", "hi Bob") == 2)  # Should match "hi" and "Bob"
# print("Test 2:", subsequence_similarity("a b c", "a c b") == 2)  # Should match "a" and either "b" or "c"
# print("Test 3:", subsequence_similarity("x y z", "a b c") == 0)  # No common subsequence except single words
# print("Test 4:", subsequence_similarity("", " ") == 0)  # Empty strings
# print("Test 5:", subsequence_similarity("hello world", "hello there world") == 2)  # "hello" and "world"

# Checks if the edit distance between l1 and l2 is below threshold*max(len(l1),len(l2))
def prefix_sum(l):
  s=0
  out=[]
  for e in l:
    s+=e
    out.append(s)
  return out
def postfix_sum(l):
  s=0
  out=[]
  for e in reversed(l):
    s+=e
    out.append(s)
  return list(reversed(out))
def edit_distance_below(l1,l2,threshold):
  if abs(len(l1)-len(l2))>threshold:
    return False
  dp=np.empty((len(l2)+1,len(l1)+1))
  dp[0]=range(len(l1)+1)
  dp[:,0]=range(len(l2)+1)
  min_dist=1
  max_num=max(len(l2),len(l1))+1
  for i in range(2*len(l2)+2):
    min_dist_last=min_dist
    min_dist=max_num
    for j in range(max(i-len(l1)-1,0),min(i+1,len(l2)+1)):
      col=i-j
      row=j
      if row > len(l2) or col > len(l1):
        continue
      if row>0 and col>0:
        dp[row,col]=min(dp[row][col-1]+1,dp[row-1][col]+1,dp[row-1][col-1] if l2[row-1]==l1[col-1] else dp[row-1][col-1]+1)
      if dp[row,col]<min_dist:
        min_dist=dp[row,col]
    if min_dist<max_num and min(min_dist,min_dist_last)>threshold:
      return False
  return dp[-1,-1]<=threshold
def test_edit_distance_below():
  # Test cases for edit_distance_below function
  # Test empty strings
  assert edit_distance_below([], [], 0) == True, "Failed on empty strings"
  # Test one empty string
  assert edit_distance_below([], ['a'], 0.5) == False, "Failed on one empty string"
  assert edit_distance_below(['a'], [], 0.5) == False, "Failed on one empty string"
  # Test single character difference
  assert edit_distance_below(['a'], ['b'], 1.0) == True, "Failed on single character difference"
  assert edit_distance_below(['a'], ['b'], 0.5) == False, "Failed on single character difference"
  # Test same strings
  assert edit_distance_below(['a', 'b', 'c'], ['a', 'b', 'c'], 1.5) == True, "Failed on same strings"
  # Test insertion
  assert edit_distance_below(['a', 'c'], ['a', 'b', 'c'], 1.5) == True, "Failed on insertion"
  assert edit_distance_below(['a', 'c'], ['a', 'b', 'c'], 0.9) == False, "Failed on insertion"
  # Test deletion
  assert edit_distance_below(['a', 'b', 'c'], ['a', 'c'], 1.5) == True, "Failed on deletion"
  assert edit_distance_below(['a', 'b', 'c'], ['a', 'c'], 0.9) == False, "Failed on deletion"
  # Test substitution
  assert edit_distance_below(['a', 'b', 'c'], ['a', 'd', 'c'], 1.5) == True, "Failed on substitution"
  assert edit_distance_below(['a', 'b', 'c'], ['a', 'd', 'c'], 0.9) == False, "Failed on substitution"
  # Test multiple operations
  assert edit_distance_below(['a', 'b'], ['c', 'd'], 2) == True, "Failed on multiple operations"
  assert edit_distance_below(['a', 'b'], ['c', 'd'], 1) == False, "Failed on multiple operations"
  # Test longer strings
  assert edit_distance_below(['a', 'b', 'c', 'd'], ['a', 'x', 'c', 'y'], 2) == True, "Failed on longer strings"
  assert edit_distance_below(['a', 'b', 'c', 'd'], ['a', 'x', 'c', 'y'], 1.3) == False, "Failed on longer strings"

  # Run the test cases
test_edit_distance_below()


In [ ]:
# If you want to process all the actions ahead of time
import json
import pandas as pd

datasets = [f"data/US/{term*2+2009-222}-{term*2+2010-222}_{term}th_Congress/csv/history.csv" for term in range(111,120)]
history_df=pd.concat([pd.read_csv(dataset) for dataset in datasets])
actions = list(set(history_df["action"]))
for action in actions:
  process_split_action(action,action_process_map) # build up memo 
json.dump(action_process_map,open("./outputs/07-23/action_process_map.json","w"),indent=2)

In [ ]:
def special_buckets_f(action):
  if re.search(r"^h\.amdt\.[0-9]*? amendment \(.*?\) in the nature of a substitute offered by",action.lower()):
    return "H.Amdt. in the nature of a substitute offered by"
  if re.search(r"^h\.amdt\.[0-9]*? amendment \(.*?\) offered by",action.lower()):
    return "H.Amdt. offered by"
  if re.search(r"^s\.amdt\.[0-9]*? amendment sa [0-9]*? proposed by",action.lower()):
    return "S.Amdt. offered by"
  if re.search(r"the.*? house.*? proceeded.*? with.*? of debate.*? on.*? the.*? amendment",action.lower()):
    return "DEBATE - The House proceeded with 10 minutes of debate on the Broun (GA) motion to recommit with instructions"
  return None

In [ ]:
print(special_buckets_f("Floor summary: DEBATE - The House proceeded with 10 minutes of debate on the Delaney motion to recommit with instructions"))

In [ ]:
print(process_action("H.AMDT.1 Amendment (A001) offered by Mr. Becerra. (consideration: CR H6; text: CR H6)Amendment nominated Democrat candidates for Officers of the House."))
print(process_action("H.AMDT.100 Amendment (A011) offered by Mr. Minnick. (consideration: CR H5029; text: CR H5029)Amendment provides that the amount of a balance following a notice of a rate increase would be protected from the rate of increase as of the 7-day mark instead of the 14-day mark."))

In [ ]:
from collections import defaultdict

buckets_saved={}
def bucket_actions(actions,similar,special_bucket_f = lambda x:None,bucket_names_f=lambda x:x):
  special_buckets=defaultdict(list)
  bucket_list=[]
  
  for i,action in enumerate(actions):
    bucketed=False
    if action_special_bucket:=special_bucket_f(action):
      special_buckets[action_special_bucket].append(action)
      bucketed=True
    else:
      for bucket in reversed(bucket_list):
        if similar(bucket[0],action):
          bucket.append(action)
          bucketed=True
          break
    if i%1000==0:
      print(i)
    if not bucketed:
      print(action)
      bucket_list.append([action])
  buckets = {bucket_names_f(bucket[0]):bucket for bucket in bucket_list}
  buckets.update(special_buckets)
  buckets_saved.update(buckets)
  # bucket_list_saved.append(bucket_list)
  return buckets

In [ ]:
# Bucket actions
import pandas as pd
import json
import re
from collections import defaultdict

# actions=
# actions = [sentence_split(action) for action in history_df["action"]]

# buckets=bucket_actions(lambda action1,action2:similar_string(action1,action2,1/7  ),special_bucket_f=special_buckets_f,bucket_names_f=lambda x:process_action(x))
datasets = [f"data/US/{term*2+2009-222}-{term*2+2010-222}_{term}th_Congress/csv/history.csv" for term in range(111,120)]
history_df=pd.concat([pd.read_csv(dataset) for dataset in datasets])
actions = sorted(list(set(history_df["action"])))
buckets=bucket_actions(actions,lambda action1,action2:similar_string(action1,action2,1/7),special_bucket_f=special_buckets_f)
bucket_names=list(buckets.keys())
# buckets=bucket_actions(lambda action1,action2:similar_words(action1,action2,1/4),special_bucket_f=special_buckets_f)
  

# buckets_raw=[]
# special_buckets_raw=[[],[],[]]
# actions = 
# for i,action in enumerate(actions):
#   bucketed=False
#   # if re.search(r"^debate - the house proceeded with 10 minutes of debate on the .*?motion to recommit",action[1].lower()):
#   #   special_buckets_raw[3].append(action)
#   #   continue
#   for bucket in reversed(buckets_raw):
#     # if action[0]==bucket[0][0]:
#     #   bucket.append(action)
#     #   bucketed=True
#     #   break
#     if abs(len(action[0])-len(bucket[0][0]))<1/3*max(len(bucket[0][0]),len(action[0])) and subsequence_similarity(action[0],bucket[0][0])>=max(len(action[0])*2/3, len(bucket[0][0])*2/3):
#       # print(action)
#       bucket.append(action)
#       bucketed=True
#       break
# buckets_raw.extend(special_buckets_raw)
# buckets = [[action[1] for action in bucket if type(action[1])==str] for bucket in buckets_raw]
# display(buckets)


In [ ]:
import json
with open("outputs/buckets_after_refinement_08-01.json","r") as file:
  buckets_refined=json.load(file)
bucket_names_refined=sorted(list(buckets_refined.keys()))

In [ ]:
display(buckets_refined)

In [ ]:
import json
with open("outputs/buckets_after_refinement_08-01.json","r") as file:
  buckets_refined=json.load(file)
bucket_names_refined=sorted(list(buckets_refined.keys()))
bucket_names_refined_combined = bucket_actions(bucket_names_refined,lambda name1, name2: edit_distance_below(name1,name2,1/7*max(len(name1),len(name2))))

In [ ]:
def different_buckets(actions,f):
  buckets=defaultdict(list)
  for action in actions:
    for bucket in f(action):
      buckets[bucket].append(action)
  return dict(buckets)
starting_regexes=[r"^All points of consideration against consideration.*? are waived",\
         r"^ANNOUNCEMENT",r"^APPOINTMENT OF CONFEREE",r"^APPOINTMENT OF.*? CONFEREE",\
          r"^forwarded by subcommittee to full committee",r"^GENERAL DEBATE",r"^Introduced in the Senate",\
          r"^MOMENT OF SILENCE",r"^Motion by \[SENATOR\] to (commit|refer) to",r"^Motion (by \[SENATOR\] )?to concur",\
          r"^Motion by \[SENATOR\] to instruct",r"^Motion by \[SENATOR\] to reconsider",r"^Motion to disagree",\
          r"^Motion to discharge",r"^Motion to proceed to consideration",r"^Motion to table the motion",
          r"^Motion to waive",r"^NOTIFICATION OF INTENT","^ORDER OF BUSINESS","^ORDER OF PROCEDURE",
          r"^On agreeing to the resolution",r"On agreeing to the \[REPRESENTATIVE\] amendment",\
          r"On motion that the House (dis)?agree to the Senate amendment",\
          r"On motion that the House agree with an amendment",\
          r"On motion that the House (dis)?agree",
          r"On motion that the House instruct conferees",
          r"^On motion (that the House |to )suspend the rules",r"^On passage",r"^On ordering the previous question",
          r"^On questioning of consideration",r"^Ordered to be [r|R]eported",r"^point of order",
          r"^POSTPONED CONSIDERATION OF VETO MESSAGE",r"^POSTPONED PROCEEDING",r"POSTPONED ROLL CALL VOTE",
          r"^Passed Senate",r"^Previous question shall be considered as ordered",\
          r"Previous question shall be considered as ordered",\
          r"^Provides for consideration of \[BILL",r"^Provides for consideration of the Senate Amendment",\
          r"^Provides for consideration", r"^Provides for( \w+)? \[TIME\] of.*? debate",\
          r"^Pursuant to \[RESOLUTION\]", r"^Pursuant to a previous( \w+)? order", r"Pursuant to clause",\
          r"^Pursuant to the order of",\
          r"^proceeded with.*? debate",r"^Pursuant to the provisions of \[RESOLUTION\]",r"^QUESTION OF CONSIDERATION",\
          r"^QUESTION OF CONSIDERATION",r"^QUESTION OF THE PRIVILEGES OF THE HOUSE",r"^Received in the Senate",\
          r"^received in the senate[\,\.] read",\
          r"^Referred",r"^Resolution agreed to in Senate",r"Resolution provides for( \w+)? \[TIME\] of general debate",\
          r"(Rule|Resolution) provides for( \w+)? \[TIME\] of general debate","debate",\
          r"^Resolution provides for consideration of( \w+)? \[BILL",r"^Resolving differences",r"^Rule provides for consideration",\
          r"^Rules Committee Resolution",r"^Ruling of the Chair",r"^Second cloture", r"See( \w+)\[BILL\] ",
          r"^Senate Committee.*? discharged",r"^Senate agreed to.*? amendment",r"^Senate agreed to conference report",\
            r"^Senate agreed to",r"^Senate appointed conferee",r"Senate concurred in", r"Senate disagree. to.*? House amendment",
            r"^Senate insists on its amendment",r"^Senate passed companion measure", r"^Senate returned papers", r"^Senate vitiated previous",
            r"^Star Print ordered",r"^The Chair announced",r"^The Chair","postponed",r"^The Committee of the Whole",
            "The Committee of the Whole proceeded",
            r"^The Committee rose informally",r"The House proceeded with.*? \[TIME\] of debate",r"^The House resolved into Committee of the Whole",
            r"^The Speaker appointed (additional )?conferees"]
other_regexes=[r"^On motion","suspend the rules"]
def special_buckets_f(action):
  out=[]
  for regex in regexes:
    if re.search(regex,action.lower() if regex.islower() else action):
      out.append(action)
  return out




In [ ]:
print(len(bucket_names_refined))
print(len(bucket_names_refined_combined))

In [ ]:
buckets_refined_combined={}
unbucketed=[]
for combination in bucket_names_refined_combined:
  buckets_refined_combined[combination]=[]
  for name in bucket_names_refined_combined[combination]:
    if name not in buckets_refined_combined:
      unbucketed.append(name)
      continue
    buckets_refined_combined[combination].extend(buckets_refined[name])


In [ ]:
with open("outputs/buckets_refined_combined_08-04.json","w") as file:
  json.dump(buckets_refined_combined,file)

In [ ]:
display(buckets_refined)

In [ ]:
display(buckets_refined_combined)

In [ ]:
# display(bucket_list_saved)

# special_buckets={buckets:bucket for name in buckets if }
buckets = {process_action(bucket[0]):bucket for bucket in bucket_list_saved[0]}
buckets.update(special_buckets)
# bucket_list_names=[process_action(bucket[0])]

# buckets.update(special_buckets)
display(buckets)

In [ ]:
# Save buckets
import json
from datetime import datetime
with open(f"./outputs/buckets_{datetime.now().strftime(f"%m-%d")}_charsim17.json","w") as file:
  json.dump(buckets,file,indent=2)
with open(f"./outputs/bucket_names_{datetime.now().strftime(f"%m-%d")}_charsim17.txt","w") as file:
  for name in bucket_names:
    file.write(name)
    file.write("\n")

In [ ]:
buckets=special_buckets
display(len(buckets))

In [ ]:
import json
with open("outputs/buckets_07-29_textsimilarity17.json","r") as file:
  buckets = json.load(file)

In [ ]:
def make_new_buckets(fs,buckets):
  out=defaultdict(list)
  for bucket_name in buckets:
    moved=False
    for f in fs:
      if f_bucket:= f(bucket_name):
        out[f_bucket].extend(bucket)
        moved=True
        break
    if not moved:
      out[bucket_name].append(action)
  out=dict(out)
  return out
def make_new_buckets(fs,buckets):
  out=defaultdict(list)
  for bucket_name in buckets:
    for action in buckets[bucket_name]:
      # print(bucket)
      moved=False
      for f in fs:
        if f_bucket:= f(action):
          out[f_bucket].append(action)
          moved=True
          break
      if not moved:
        out[bucket_name].append(action)
  out=dict(out)
  return out



# buckets_manual=combine_buckets(lambda bucket:re.search(r"^h\.amdt\.[0-9]*? amendment \(.*?\) in the nature of a substitute offered by",bucket[0].lower()) if type(bucket)==list and len(bucket)>0 else False,buckets_manual)

# buckets_manual=combine_buckets(lambda bucket:re.search(r"^h\.amdt\.[0-9]*? amendment \(.*?\) offered by",bucket[0].lower()) if type(bucket)==list and len(bucket)>0 else False,buckets_manual)
# buckets_manual=combine_buckets(lambda bucket:re.search(r"^s\.amdt\.[0-9]*? amendment sa [0-9]*? proposed by",bucket[0].lower()) if type(bucket)==list and len(bucket)>0 else False,buckets_manual)


In [ ]:
specific_action_to_bucket_name={}
for bucket in buckets:
  for action in bucket:
    specific_action_to_bucket_name[action]=bucket[0]
# display(specific_action_to_bucket_name)

In [ ]:
display(buckets)

In [ ]:
# bucket_map={action:bucket for bucket in buckets for action in buckets_saved[bucket]}
bucket_map={}
for name in buckets:
  for action in buckets[name]:
    bucket_map[action]=name
# display(bucket_map)
bucket_sizes={name:0 for name in buckets}
unbucketed=[]
bucketed=[]
for action in history_df["action"]:
  if action not in bucket_map:
    unbucketed.append(action)
    continue
  bucketed.append(action)
  bucket_sizes[bucket_map[action]]+=1


In [ ]:
display(new_buckets)

In [ ]:
buckets.update(new_buckets)

In [ ]:
display(sorted(set(unbucketed)))
display(len(unbucketed))
display(len(bucketed))

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
def nums_to_bars(nums):
  counter = Counter(nums)
  keyvals=counter.items()
  print(sorted(keyvals))

  return [key for key,val in keyvals],[val for key,val in keyvals]
xs,heights = nums_to_bars(bucket_sizes.values())
ax.bar(xs,heights)
ax.loglog()
# ax.set_ybound(0,100)
# ax.set_xbound(0,100)
fig.show()

In [ ]:
bucket_names = [bucket[0] for bucket in buckets if type(bucket)==list and len(bucket)>0]
datasets = [f"data/US/{term*2+2009-222}-{term*2+2010-222}_{term}th_Congress/csv/history.csv" for term in range(111,120)]
history_df=pd.concat([pd.read_csv(dataset) for dataset in datasets])
history_df["bucket"]=history_df["action"].apply(lambda action:specific_action_to_bucket_name[action])
llm_buckets = []
with open("./outputs/bucket_names_output.jsonl", "r") as file:
  for line in file:
    if line.strip():  # Skip empty lines
      response = json.loads(line)
      llm_buckets.append(response["response"]["body"]["output"][0]["content"][0]["text"])
      
history_df["llm_bucket"] = history_df["bucket"].map(dict(zip(bucket_names, llm_buckets)))

# Add next bill_id column to compare
history_df["next_bill_id"]=history_df["bill_id"].shift(-1)
history_df["next_bucket"]=history_df["bucket"].shift(-1)
history_df["next_bucket"]=history_df.apply(lambda row:row["next_bucket"] if row["next_bill_id"]==row["bill_id"] else None,1)
history_df["next_bucket_llm"]=history_df["llm_bucket"].shift(-1)
history_df["next_llm_bucket"]=history_df.apply(lambda row:row["next_bucket_llm"] if row["next_bill_id"]==row["bill_id"] else None,1)

g={}
for bucket1 in llm_buckets:
  g[bucket1]={}
  for bucket2 in llm_buckets:
    g[bucket1][bucket2]=0
  g[bucket1][None]=0
for i, row in history_df.iterrows():
  g[row["llm_bucket"]][row["next_llm_bucket"]]+=1
# display(g)



# display(history_df)
# for i,row in history_df.iterrows():
#   print(row.bucket)
# for action in history_df:
#   history_df

In [ ]:
for bucket1 in llm_buckets:
  for bucket2 in llm_buckets:
    if g[bucket1][bucket2]>200:
      print(bucket1+" -> "+bucket2)

In [ ]:
# Prepare batch to come up with names for each bucket
import random
import numpy as np
bucket_names_to_bucket={bucket[0]:bucket for bucket in buckets if type(bucket)==list and len(bucket)>0}
manual_name_to_llm_reponse={}
random.seed(1423)
bucket_names_str="\n".join(random.sample(bucket_names,k=20))
BATCHES=4
bucket_names_batches=np.split(np.array(bucket_names),BATCHES)
for batch_i in range(BATCHES):
  input_len=0
  with open(f"./outputs/bucket_names_batch{batch_i}_input.jsonl","w") as file:
    for i,bucket_name in enumerate(bucket_names_batches[batch_i]):
      input=f"""I am analyzing a dataset full of Congressional actions. I have categorized these actions into buckets,\
      where each bucket has near-identical actions, usually differing only in the names/numbers of congresspeople,bills,votes,and committees. \
      I want to come up with names for these buckets. These names are usually just the actions with names/numbers of congresspeople, bills, votes and committees removed.
      First, here is a sample of one action from the 20 buckets chosen at random:\n
      {bucket_names_str}\n
      Here are some example names you might give to bucket actions:\n
      Action from bucket: Mr. Whitfield moved that the Committee now rise.\n
      Bucket name: Congressperson moved the Committee now rise.\n\n
      Action from bucket: The House resumed with the motion to agree in the Senate amendment to H.R. 83, with an amendment. (consideration: CR H9284-9290)\n
      Bucket name: The House resumed with the motion to agree in the Senate amendment to the bill, with an amendment.\n\n
      Action from bucket: Subcommittee Consideration and Mark-up Session Held and Forwarded to Full Committee by the Subcommittee on Capital Markets and Government Sponsored Enterprises Prior to Introduction and Referral Revised discussion draft, as amended, ordered favorably reported to the full committee by the subcommittee on Capital Markets and Government Sponsored Enterprises\n
      Bucket name: Subcommittee Consideration and Mark-up Session Held and Forwarded to Full Committee by the Subcommittee Prior to Introduction and Referral. Revised discussion draft, as amended, ordered favorably reported to the full committee by the subcommittee\n\n
      Action from bucket: Passed Senate with amendments by Yea-Nay. 93 - 7. Record Vote Number: 218.\n
      Bucket name: Passed Senate with amendments by Yea-Nay.\n\n
      
      Now, please come up with a name for the bucket with the following actions. Please only give me the name, nothing else in your response.
      If there is only one line, it means the bucket has only one action:\n
      {"\n".join(random.sample(bucket_names_to_bucket[bucket_name],k=20) if len(bucket_names_to_bucket[bucket_name])>20 else bucket_names_to_bucket[bucket_name])}"""
    
      input_len+=len(input)
      json.dump({"custom_id":"action"+str(i),"url":"/v1/responses","method":"POST","body":{"input":input,"model":"gpt-4.1-mini"}}, file)
      file.write("\n")
    # manual_name_to_llm_reponse[bucket_name]=client.responses.create(input=input,model="gpt-4.1-mini")
    # print(bucket_name)
  print(input_len)

In [ ]:

import dotenv
import os
import openai
import asyncio
import json
dotenv.load_dotenv()
client = openai.Client(api_key=os.environ["OPENAI_API_KEY"])
for bucket_name in bucket_names[:5]:
      input=f"""I am analyzing a dataset full of Congressional actions. I have categorized these actions into buckets,\
      where each bucket has near-identical actions, usually differing only in the names/numbers of congresspeople,bills,votes,and committees. \
      I want to come up with names for these buckets. These names are usually just the actions with names/numbers of congresspeople, bills, votes and committees removed.
      First, here is a sample of one action from the 20 buckets chosen at random:\n
      {bucket_names_str}\n
      Here are some example names you might give to bucket actions:\n
      Action from bucket: Mr. Whitfield moved that the Committee now rise.\n
      Bucket name: Congressperson moved the Committee now rise.\n\n
      Action from bucket: The House resumed with the motion to agree in the Senate amendment to H.R. 83, with an amendment. (consideration: CR H9284-9290)\n
      Bucket name: The House resumed with the motion to agree in the Senate amendment to the bill, with an amendment.\n\n
      Action from bucket: Subcommittee Consideration and Mark-up Session Held and Forwarded to Full Committee by the Subcommittee on Capital Markets and Government Sponsored Enterprises Prior to Introduction and Referral Revised discussion draft, as amended, ordered favorably reported to the full committee by the subcommittee on Capital Markets and Government Sponsored Enterprises\n
      Bucket name: Subcommittee Consideration and Mark-up Session Held and Forwarded to Full Committee by the Subcommittee Prior to Introduction and Referral. Revised discussion draft, as amended, ordered favorably reported to the full committee by the subcommittee\n\n
      Action from bucket: Passed Senate with amendments by Yea-Nay. 93 - 7. Record Vote Number: 218.\n
      Bucket name: Passed Senate with amendments by Yea-Nay.\n\n

      Now, please come up with a name for the bucket with the following actions. Please only give me the name, nothing else in your response.
      If there is only one line, it means the bucket has only one action:\n
      {"\n".join(random.sample(bucket_names_to_bucket[bucket_name],k=20) if len(bucket_names_to_bucket[bucket_name])>20 else bucket_names_to_bucket[bucket_name])}"""
      response=client.responses.create(input=input,model="gpt-4.1-mini")
      print(response.output_text)

In [ ]:
print(response.output_text)

In [ ]:
# Load OpenAI client
import dotenv
import os
import openai
import asyncio
import json
import time
dotenv.load_dotenv()
client = openai.Client(api_key=os.environ["OPENAI_API_KEY"])
statuses=[batch.status for batch in client.batches.list()]
while "cancelling" in statuses or "in_progress" in statuses:
  time.sleep(60)
  statuses=[batch.status for batch in client.batches.list()]
print("starting batches")
for batch_i in range(3,4):
  with open(f"./outputs/bucket_names_batch{batch_i}_input.jsonl","rb") as file:
    batch_file=client.files.create(file=file,purpose="batch")
  batch=client.batches.create(input_file_id=batch_file.id,endpoint="/v1/responses",completion_window="24h")
  while batch.status!="completed":
    if batch.status=="failed":
      break
    time.sleep(60)
  with open(f"./outputs/bucket_names_batch{batch_i}_output.jsonl","w") as file:
    file.write(client.files.content(file_id=client.batches.retrieve(batch_id=batch.id).output_file_id).text)
  print(f"finished batch {batch_i}")
  time.sleep(60)

In [ ]:
with open(f"./outputs/bucket_names_batch3_output.jsonl","w") as file:
  file.write(client.files.content(file_id=client.batches.retrieve(batch_id="batch_687e820d9b448190be9761b57efa014d").output_file_id).text)

In [ ]:
with open("./outputs/bucket_names_output.jsonl", "w") as outfile:
  for batch_i in range(4):
    with open(f"./outputs/bucket_names_batch{batch_i}_output.jsonl", "r") as infile:
      outfile.write(infile.read())

In [ ]:
print(batch.id)

In [ ]:
client.batches.cancel(batch_id=batch.id)

In [ ]:
client.batches.retrieve(batch_id=batch.id)



In [ ]:
import json
with open("./outputs/buckets_manual3.json","w") as file:
  json.dump(buckets_manual,file)
  # for bucket in buckets:
  #   file.write(str(bucket))
  # file.writelines(buckets_manual)
# display([bucket for bucket in sorted(buckets_manual, key=lambda x:len(x),reverse=True)])

In [ ]:
import openai
bucket_name_to_bucket={bucket[0]:bucket for bucket in buckets_manual if type(bucket)==list and len(bucket)>0}
for bucket in bucket_name_to_bucket:



In [ ]:
display(len(set(history_df["action"])))

In [ ]:
llm_buckets=[[] for bucket in buckets]
misclassified_by_subsequence=[]
responses=[]
for i in range(13,len(buckets)):
  bruteforce_bucket=buckets[i]
  print(i,bruteforce_bucket[0])
  responses=[]
  async with asyncio.TaskGroup() as tg:
    for action in bruteforce_bucket:
      input = "These are two actions taken from a dataset of Congressional Actions. Please tell me if these two actions are the same except for potentially different names/numbers of committees, senators, representatives, bills, motions, or amendments, and potentially different vote records as well. Give me a one word answer \"Yes\" or \"No\":\n"\
        + action+"\n"+bruteforce_bucket[0]
      print(action)
      responses.append((action,tg.create_task(client.responses.create(input=input,model="gpt-4.1-nano"))))
  for response in responses:
    if response[1].result().output_text == "Yes":
      llm_buckets[i].append(response[0])
    else:
      misclassified_by_subsequence.append(action)

In [ ]:
)

In [ ]:
import pandas as pd
import re
from collections import defaultdict, Counter
datasets = [f"data/US/{term*2+2009-222}-{term*2+2010-222}_{term}th_Congress/csv/history.csv" for term in range(111,120)]
history_df=pd.concat([pd.read_csv(dataset) for dataset in datasets])

# bills_df_updated = bills_df.copy(   )
actions=set()
referred=set()
ldict = {"committee":{},"general":[],"Submitted in the Senate":[],"Introduced in House":[]}
subcommittee_dict={"misc":[]}
committee_cats=[]


committee_dict=defaultdict(list)
# for regex in committee_cats:
#   subcommittee_dict[regex]=[]
committee_regexes = [
  r'house rose',
  r'committee rose',
  r'moved that the committee rise',
  r'committee rise agreed to by voice vote',
  r'Reported \(Amended\) by',
  r'submitted in the senate, considered, and agreed to',
  r'motion by senator',
  r'Committee of the Whole House on the state of the Union',
  r'reported an original measure',
  r'Ordered to be reported .*? favorably',
  r'Original measure reported to Senate',
  r'the speaker designated the honorable',
  r'Granted an extension for further consideration',
  r'motion to discharge committee filed',
  r'withdrawn by unanimous consent',
  r'asked unanimous consent',
  r'motion to refer the bill',
  r'the speaker appointed [conferees|additional conferees]',
  r'filed written report',
  r'motion to discharge senate committee',
  r'the committee substitute as amended agreed to by unanimous consent',
  r'the committee amendment as amended agreed to by unanimous consent',
  r'supplemental report filed by',
  r'Pursuant to clause 6\(h\) of rule XVIII, the Committee of the Whole resumed its sitting',
  r'by direction of .*? and asked for its immediate consideration',
  r'motion to discharge the committee on rules',
  r'motion to table the motion',
  r'reported adversely.*? by the committee',
  r'rules committee resolution .*? reported to house',
  r'on motion that the committee now rise agreed to .*?[by voice vote|without objection]',
  r'failed to report favorably',
  r'ordered to be reported favorably',
  r'The resolution provides? that.*? amendment in the nature of a substitute.*? shall be considered as adopted',
  r'committee amendments?.*? agreed to by unanimous consent',
  r'moved to commit to the committee',
  r'committee agreed to seek consideration under suspension of the rules',
  r'all points of order against consideration .*? are waived',
  "committee consideration and mark-up session held",
  "hearings held",
  r"committee.*? discharged",
  r"reported by.*?with.*?report",
  r"reported by.*?without.*?report",
  "forwarded by subcommittee to full committee.*? by voice vote",
  "forwarded by subcommittee to full committee.*? by unanimous consent",
  "forwarded by subcommittee to full committee.*? by the yeas and nays",
  "forwarded by subcommittee to full committee.*? in the nature of a substitute .*?by the yeas and nays",
  "forwarded by subcommittee to full committee.*? in the nature of a substitute .*?by voice vote",
  "forwarded by subcommittee to full committee.*? in the nature of a substitute .*?by unanimous consent",
  r"amendment.*? offered by",
  "it shall be in order to consider as an original bill for the purpose of amendment",
  "moved to recommit",
  "moved that the committee now rise",
  r"[read twice and referred to]|[read the second time and referred to]",
  r"^reported by",
  r"amdt.*?referred to",
  r"amendment.*? proposed by",
  r"the committee resumed? it\'?s sitting",
  r"by[| the] direction of the committee on rules,.*? called up",
  "rule provides for consideration of",
  "resolution provides for consideration of",
  r"^referred",
  r"^re-referred",
  "^received in the senate and referred"
]
for row in history_df.itertuples():
  # print(row.action)
  # match = re.match(r'^referred to the (house )?(committee|subcommittee)', row.action.lower())
  categorized=False
  if re.search("committee",row.action.lower()) and not re.search("committee of the whole",row.action.lower()):
    for regex in committee_regexes:
      if re.search(regex,row.action.lower() if regex.islower() else row.action):
        committee_dict[regex].append(row.action)
        categorized=True
        break
    if not categorized:
      committee_dict["misc"].append(row.action)
  elif re.search("Submitted in the Senate",action):
    ldict["Submitted in the Senate"].append(action)
  elif re.match(r"Introduced in House$",action):
    ldict["Introduced in House"].append(action)
  else:
    ldict["general"].append(action)
# committee_dict=dict(committee_dict)
display(sorted(committee_dict["misc"]))
# display(ldict["general"])
# display(ldict["committee"])
# for key in ldict:
#   display(ldict[key])
# display(ldict)
# for key in types:
#   print(key,types[key])


# bills_df["passed"] = bills_df[]
# history_df['referred to committee'] = history_df['action'].str.lower().str.match(r'^referred to the (house committee|committee|house subcommittee|subcommittee)')


In [ ]:
print(not re.search(r"general debate","GENERAL DEBATE - The Committee of the Whole proceeded with one hour of general debate on H.R. 277.".lower()))

In [ ]:
if "Reported (Amended) by" in "Reported (Amended) by the Committee on Natural Resources. H. Rept. 118-919.":
  print("hi")

In [ ]:
from openai import OpenAI
import os
import dotenv
dotenv.load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [ ]:


batch_input_file = client.files.create(
    file=open("embedding_batch.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file)

In [ ]:
batch = client.batches.create(input_file_id=batch_input_file.id,endpoint="/v1/outputs",completion_window="24h",metadata={"description":"nightly eval job"})

In [ ]:
for obj in client.files.content("file-Nybj7TyzxjuL7NXTWsNqGw").text.split("\n"):
  print(json.loads(obj))

In [ ]:
with open("./outputs/bucket_names_batch0_output.jsonl","w") as file:
  file.write(client.files.content("file-Nybj7TyzxjuL7NXTWsNqGw").text)

In [ ]:
display(list(client.files.retr))

In [ ]:
display(client.batches.retrieve("batch_685462b110ac8190bb51093fa9c12910"))

In [ ]:
input_file = client.files.content("file-Vt5fNxGnSo2qh796HC27PC")

In [ ]:
# file_response = client.files.content("file-Vt5fNxGnSo2qh796HC27PC")
# with open("outputs/2025-26-history.jsonl","w") as f:
#   f.write(file_response.text)
json_file = pd.read_json("outputs/2025-26-history.jsonl",lines=True)

In [ ]:
import pandas as pd
import sklearn
import sklearn.cluster
import sklearn.manifold
import numpy as np

print(json_file.iloc[0])
df = pd.DataFrame()
df["bill#"]=json_file["custom_id"]
# df["bill#"]=json_file.apply(lambda x:x["custom_id"])
df["embedding"]=json_file["response"].apply(lambda x: x["body"]["data"][0]["embedding"])
df.sort_values(by="bill#")
outputs_array = np.array(df["embedding"].tolist())
# tsne_result = sklearn.manifold.TSNE().fit_transform(embeddings_array)
# display(df)
# display
# (json_file["response"].iloc[0])
# display(type(json_file["custom_id"]))
# json_file.iloc[0]["response"]["embedding"]

In [ ]:
import json
history_df = pd.read_csv("data/US/2025-2026_119th_Congress/csv/history.csv")
def bucket(cluster_per_action):
  clusters={}
  for i,cluster in enumerate(cluster_per_action):
    cluster=int(cluster)
    if cluster not in clusters:
      clusters[cluster]=set()
    clusters[cluster].add(history_df.iloc[i]["action"])
  return list(clusters.values())
def unique_actions(buckets):
  buckets_unique=[]
  for bucket in buckets:
    bucket_unique={re.sub("-","",re.sub(r'\d+', '', x)) for x in bucket}
    buckets_unique.append(list(bucket_unique))
  return buckets_unique



In [ ]:
# optics_result = sklearn.cluster.OPTICS().fit_predict(embeddings_array)
# optics_buckets = unique_actions(bucket(optics_result))
# display(optics_buckets)

In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np
topleft_pts=[i for i,x in enumerate(tsne_result) if x[0]<-90 and x[1]>0]
input_file=[json.loads(x)["body"]["input"] for x in open("embedding_batch.jsonl", "rb")]

def plot_2d(arr):
  plt.scatter([x[0] for x in arr],[x[1] for x in arr])
  plt.show()



intro_house=[i for i in range(len(input_file)) if input_file[i]=="Introduced in House"]
# np.mean()
means_overall=np.mean(embeddings_array,axis=0)
std_overall = np.std(embeddings_array,axis=0)
means_house=np.mean(embeddings_array[intro_house],axis=0)
std_house = np.std(embeddings_array[intro_house],axis=0)
print(np.mean(std_overall))
print(np.mean(std_house))
# fig, axs = plt.subplots(2)
# axs[0].bar(range(len(means_overall)),means_overall-means_house)
# ax[1].bar(range(len(means_overall)),std_overall)
# display(np.std(embeddings_array[intro_house]))
# display([input_file[x] for x in topleft_pts])


In [ ]:
print(results[1])